In [1]:
import os              
os.environ['PYTHONHASHSEED'] = '0'
import pandas as pd                                                    
import numpy as np                                                     
import scanpy as sc                                                                                 
from time import time                                                       
import sys
import matplotlib
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from anndata import AnnData, read_h5ad, concat
from tqdm import tqdm
import scipy
import scipy.stats as ss

In [2]:
import pandas as pd
from openpyxl import load_workbook
# from xlrd import open_workbook

def read_xlsx(file, sheet_name=None, header=None):
    excel = pd.ExcelFile(load_workbook(file), engine="openpyxl")
    sheet_name = sheet_name or excel.sheet_names[0]
    sheet = excel.book[sheet_name]
    df = excel.parse(sheet_name, header=header)

    for item in sheet.merged_cells:
        top_col, top_row, bottom_col, bottom_row = item.bounds
        base_value = item.start_cell.value
        top_row -= 1
        top_col -= 1
        if header is not None:
            top_row -= header + 1
            bottom_row -= header + 1
        df.iloc[top_row:bottom_row, top_col:bottom_col] = base_value
    return df

In [ ]:
gw15 = read_h5ad("../source_data/gw15.h5ad"); gw15 = gw15.raw.to_adata()
gw20 = read_h5ad("../source_data/gw20.h5ad"); gw20 = gw20.raw.to_adata()
gw22 = read_h5ad("../source_data/gw22.h5ad"); gw22 = gw22.raw.to_adata()
gw34 = read_h5ad("../source_data/gw34.h5ad"); gw34 = gw34.raw.to_adata()

gw15.obs['H3_annotation'] = gw15.obs['H3_annotation'].astype(str).where(gw15.obs['H3_annotation'].notna(), gw15.obs['H2_annotation'].astype(str) + '-c0')
gw20.obs['H3_annotation'] = gw20.obs['H3_annotation'].astype(str).where(gw20.obs['H3_annotation'].notna(), gw20.obs['H2_annotation'].astype(str) + '-c0')
gw22.obs['H3_annotation'] = gw22.obs['H3_annotation'].astype(str).where(gw22.obs['H3_annotation'].notna(), gw22.obs['H2_annotation'].astype(str) + '-c0')
gw34.obs['H3_annotation'] = gw34.obs['H3_annotation'].astype(str).where(gw34.obs['H3_annotation'].notna(), gw34.obs['H2_annotation'].astype(str) + '-c0')

In [ ]:
def preprocess(adata):
    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)
    sc.pp.scale(adata, max_value=6)
    return adata    

gw15 = preprocess(gw15)
gw20 = preprocess(gw20)
gw22 = preprocess(gw22)
gw34 = preprocess(gw34)

In [ ]:
def extract_region(gw):
    gw.obs['region'] = gw.obs['region'].str[0]
    gw = gw[gw.obs['region'].isin(['F', 'P', 'O'])]
    return gw

gw15 = extract_region(gw15)
gw20 = extract_region(gw20)
gw22 = extract_region(gw22)
gw34 = extract_region(gw34)

In [7]:
en_group = read_xlsx("EN groups based on sankey.xlsx", header=0)

def rep_cluster(name):
    return name.replace(" cluster ", "-c")

key_lst = ["clusters from GW15", "clusters from GW20", "clusters from GW22", "clusters from GW34"]
data_lst = ["gw15", "gw20", "gw22", "gw34"]
# key_lst = ["clusters from GW34"]
# data_lst = ["gw34_raw"]
for i in range(len(key_lst)):
    key = key_lst[i]
    group_dict = {rep_cluster(en_group[key][i]): en_group['Group name'][i] for i in range(en_group.shape[0]) if not pd.isna(en_group[key][i]) }
    locals()[data_lst[i]].obs['group'] = locals()[data_lst[i]].obs['H3_annotation'].map(group_dict)
    locals()[data_lst[i]] = locals()[data_lst[i]][~pd.isna(locals()[data_lst[i]].obs['group'])]
    

In [8]:
gw_scale = concat([gw15, gw20, gw22, gw34], axis=0)

In [9]:
def extract_group(gw, group):
    if group == "'EN-IT-L5/6":
        gw15_sub = gw[(gw.obs.H3_annotation.isin(["EN-IT-L5/6-c1", "EN-IT-L5/6-c2",
                                       "EN-IT-L5/6-c3", "EN-IT-L5-c1",
                                       "EN-IT-L5-c2", "EN-IT-L5-c3"]))
                   & (gw.obs.gw == '15')]
        gw_other = gw[(gw.obs.group == group) & (gw.obs.H1_annotation != '15')]
        gw_group = concat([gw15_sub, gw_other], axis=0)
    elif group == "'EN-IT-L4":
        gw15_sub = gw[(gw.obs.H3_annotation.isin(["EN-IT-L3/4-1-c1"]))
                    & (gw.obs.gw == '15')]
        gw_other = gw[(gw.obs.group == group) & (gw.obs.H1_annotation != '15')]
        gw_group = concat([gw15_sub, gw_other], axis=0)
    elif group == "'EN-IT-L3":
        gw15_sub = gw[(gw.obs.H3_annotation.isin(["EN-IT-L3/4-1-c1", "EN-IT-L5/6-c1"]))
                    & (gw.obs.gw == '15')]
        gw_other = gw[(gw.obs.group == group) & (gw.obs.H1_annotation != '15')]
        gw_group = concat([gw15_sub, gw_other], axis=0)
    elif group == "'EN-IT-L2":
        gw15_sub = gw[(gw.obs.H3_annotation.isin(["EN-IT-L3/4-1-c1"]))
                    & (gw.obs.gw == '15')]
        gw_other = gw[(gw.obs.group == group) & (gw.obs.H1_annotation != '15')]
        gw_group = concat([gw15_sub, gw_other], axis=0)
    else:
        gw_group = gw[gw.obs.group == group]
    return gw_group

In [10]:
area_lst = ['F', 'P', 'O']
def zs_calcu(gw, group):
    gw_group = extract_group(gw_scale, group)
    gw_mean_lst = []
    for area in area_lst:
        gw_zs = gw_group[ (gw_group.obs['gw'] == str(gw)) & (gw_group.obs['source'] == area)]
        gw_mean = np.mean(gw_zs.X, axis=0)
        gw_mean_lst.append(gw_mean)
    gw_mean_lst = np.array(gw_mean_lst)
    return gw_mean_lst

In [14]:
os.makedirs("result/figs8", exist_ok=True)

group_lst = ["'EN-ET-L5/6", "'EN-ET-SP/L6b", "'EN-IT-L2", "'EN-IT-L3", "'EN-IT-L4", "'EN-IT-L5/6"]

for i in range(len(group_lst)):
    group = group_lst[i]
    gw15_zs = zs_calcu(15, group)
    gw20_zs = zs_calcu(20, group)
    gw22_zs = zs_calcu(22, group)
    gw34_zs = zs_calcu(34, group)
    gw_zs = pd.DataFrame(np.concatenate([gw15_zs, gw20_zs, gw22_zs, gw34_zs], axis=0))
    gw_zs.columns = gw_scale.var.index
    gw_zs['area'] = area_lst * 4
    gw_zs['gw'] = np.repeat(['gw15', 'gw20', 'gw22', 'gw34'], 3)
    group_name = group
    group_name = group_name[1:len(group_name)]
    if "/" in group_name:
        group_name = group_name.replace('/','|')
    gw_zs.to_csv(f"result/figs8/{group_name}_zs.csv")

/Users/shunzhou/opt/anaconda3/lib/python3.9/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/Users/shunzhou/opt/anaconda3/lib/python3.9/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/Users/shunzhou/opt/anaconda3/lib/python3.9/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/Users/shunzhou/opt/anaconda3/lib/python3.9/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/Users/shunzhou/opt/anaconda3/lib/python3.9/site-packages/anndata/_core/anndata.py:1828: UserWarning